# Chromatographic Processes - Exercise

## Exercise 1: Set up the non-binding experiment from the lab book

## Exercise 2: Set up the Bind and Elute experiment from the lab book

In [ ]:
# Process
process = Process(flow_sheet, 'batch elution')

process.cycle_time = 60 * 60

c_salt_load = 50
c_load = [c_salt_load, 1, 1, 1]

process.add_event('load', 'flow_sheet.inlet.c', c_load, 0)

simulator = Cadet()
simulator.time_resolution = 2


simulation_results = simulator.simulate(process)
print(simulation_results.time_elapsed)

from CADETProcess.plotting import SecondaryAxis

sec = SecondaryAxis()
sec.components = ['Salt']
sec.y_label = '$c_{salt}$'

simulation_results.solution.outlet.outlet.plot(secondary_axis=sec)
import matplotlib.pyplot as plt

plt.tight_layout()